# <i>CatBoost learning to rank</i>

In [2]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os
import pandas as pd

### Download part of [MSRank](https://www.microsoft.com/en-us/research/project/mslr/) dataset from CatBoost datasets storage

In [3]:
# Train dataset
train_df = pd.DataFrame(columns=[x for x in range(247)])
cur_len = 0

for line in open('imat2009_new_split/imat2009_train_new.txt'):
    line = line.split()
    new_row = {0:float(line[0]), 1:int(line[-1])}

    for data in line[1:-2]:
        data = data.split(':')
        new_row[int(data[0]) + 1] = float(data[1])

    train_df = train_df.append(new_row, ignore_index=True)

train_df = train_df.fillna(0)
train_df[1] = train_df[1].astype(int)

In [4]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,246
0,1.0,3382,0.000023,0.0,0.000000,0.000000,0.000000,0.0,0.704953,0.550315,...,0.032749,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000023,1.000000,0.000023
1,1.0,3382,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.273423,0.000000,...,0.032749,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.862745,0.000000
2,1.0,3382,0.000000,0.0,0.006800,0.051546,0.000000,0.0,0.671346,0.000000,...,0.032749,0.0,0.0,0.0,0.154346,0.0,0.000000,0.000000,0.811765,0.000000
3,1.0,3382,0.000000,0.0,0.000862,0.030928,0.000000,0.0,0.573946,0.000000,...,0.032749,0.0,0.0,0.0,0.039509,0.0,0.000000,0.000000,1.000000,0.000000
4,1.0,3382,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.261436,0.000000,...,0.032749,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.882353,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77709,4.0,19724,0.002004,1.0,0.194805,0.262626,0.297467,0.0,0.145516,0.690807,...,0.032749,1.0,0.0,0.0,0.000000,0.0,0.329412,0.002004,0.815686,0.002059
77710,0.0,19724,0.000016,1.0,0.003837,0.000000,0.000000,0.0,0.601876,0.421231,...,0.032749,1.0,0.0,0.0,0.000000,0.0,0.929412,0.000016,1.000000,0.000016
77711,0.0,19724,0.000000,1.0,0.020070,0.000000,0.000000,0.0,0.655321,0.000000,...,0.032749,1.0,0.0,0.0,0.265364,0.0,0.988235,0.000000,1.000000,0.000000
77712,2.0,19724,0.000015,1.0,0.000000,0.000000,0.023174,0.0,0.664482,0.561835,...,0.032749,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000015,0.862745,0.000015


In [5]:
# Test dataset
test_df = pd.DataFrame(columns=[x for x in range(247)])
cur_len = 0

for line in open('imat2009_new_split/imat2009_test_new.txt'):
    line = line.split()
    new_row = {0:float(line[0]), 1:int(line[-1])}

    for data in line[1:-2]:
        data = data.split(':')
        new_row[int(data[0]) + 1] = float(data[1])

    test_df = test_df.append(new_row, ignore_index=True)

test_df = test_df.fillna(0)
test_df[1] = test_df[1].astype(int)

In [ ]:
test_df

In [6]:
X_train = train_df.drop([0, 1], axis=1).values
y_train = train_df[0].values
queries_train = train_df[1].values

X_test = test_df.drop([0, 1], axis=1).values
y_test = test_df[0].values
queries_test = test_df[1].values


### Dataset analysis

__Number of documents__

In [7]:
num_documents = X_train.shape[0]
print(num_documents)

77714


__Number of features__

In [8]:
X_train.shape[1]

245

__Relevance labels statistics__

0 - irrelevant, 1 - highly relevant. Table represents number of documents for each value.

In [9]:
from collections import Counter
Counter(y_train).items()

dict_items([(1.0, 20086), (0.0, 25776), (2.0, 24424), (4.0, 952), (3.0, 1744), (0.5, 1982), (1.5, 1033), (0.25, 77), (1.33333, 110), (1.2, 3), (2.37037, 39), (0.666671, 340), (2.33333, 79), (0.333329, 268), (2.16049, 19), (2.5, 337), (2.87037, 26), (1.66667, 107), (2.12037, 4), (2.25, 19), (2.24074, 25), (0.2, 10), (1.6, 6), (0.8, 5), (0.6, 10), (0.875, 1), (2.66667, 31), (3.1625, 2), (1.75, 12), (0.75, 55), (2.61111, 4), (0.222229, 1), (0.4, 5), (1.25, 23), (1.97143, 2), (3.5, 16), (2.24691, 10), (2.16667, 1), (1.95239, 1), (1.4, 4), (3.66667, 5), (3.8, 2), (0.125, 1), (2.05556, 2), (3.33333, 4), (2.2, 5), (2.58025, 2), (1.16667, 2), (2.91358, 1), (2.07407, 3), (2.11729, 1), (3.25, 1), (2.375, 1), (3.21666, 1), (2.74074, 5), (2.12346, 3), (0.166671, 8), (0.833329, 5), (1.14286, 1), (3.53, 1), (3.4, 1), (2.75, 1), (3.58125, 1), (2.40741, 1), (0.583329, 1), (1.8, 1), (2.42857, 1), (2.0463, 1), (1.77143, 1), (3.75, 1), (0.888886, 1)])

For calculation such metrics as NDCG and PFound relevances should be in segment \[0,1\].

In [10]:
max_relevance = np.max(y_train)
y_train /= max_relevance
y_test /= max_relevance

__Number of queries__

In [11]:
num_queries = np.unique(queries_train).shape[0]
num_queries

7300

### Creation of CatBoost pools

In [12]:
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

### You can also create pools from files

In [13]:
data_dir = './msrank'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

train_file = os.path.join(data_dir, 'train.csv')
test_file = os.path.join(data_dir, 'test.csv')

train_df.to_csv(train_file, index=False, header=False)
test_df.to_csv(test_file, index=False, header=False)

In [14]:
description_file = os.path.join(data_dir, 'dataset.cd')
with open(description_file, 'w') as f:
    f.write('0\tLabel\n')
    f.write('1\tQueryId\n')

In [15]:
Pool(data=train_file, column_description=description_file, delimiter=',')

### <span style="color:#ce2029">Attention:</span> all objects in dataset must be grouped by group_id

For example, if the dataset consits of five documents 
\[d1, d2, d3, d4, d5\] with corresponding queries \[q1, q2, q2, q1, q2\] then the dataset should be look like:

$$\begin{pmatrix}
    d_1, q_1, f_1\\
    d_4, q_1, f_4\\
    d_2, q_2, f_2\\
    d_3, q_2, f_3\\
    d_5, q_2, f_5\\
\end{pmatrix} \hspace{6px} \texttt{or} \hspace{6px}
\begin{pmatrix}
    d_2, q_2, f_2\\
    d_3, q_2, f_3\\
    d_5, q_2, f_5\\
    d_1, q_1, f_1\\
    d_4, q_1, f_4\\
\end{pmatrix}$$

where $f_i$ is feature vector of i-th document.

### Reducing problem to machine learning task

The first and simplest idea is to try predicting document relevance $l_q$ minimizing RMSE.

$$\frac{1}{N}\sqrt{ \sum_q \sum_{d_{qk}} \left(f(d_{qk}) - l_{qk} \right)^2 }$$

In [16]:
default_parameters = {
    'iterations': 2000,
    'custom_metric': ['NDCG'],
    'verbose': False,
    'random_seed': 0,
}

parameters = {}

In [17]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

Lets train the simplest model and also demonstrate precision/recall metrics from introduction.

In [18]:
model = fit_model('RMSE', {'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']})

/Users/azamatshora/opt/anaconda3/lib/python3.8/site-packages/catboost/core.py:6240: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Group weights parameter
Suppose we know that some queries are more important than others for us.<br/>
The word "importance" used here in terms of accuracy or quality of CatBoostRanker prediction for given queries.<br/>
You can pass this additional information for learner using a ``group_weights`` parameter.<br/>
Under the hood, CatBoostRanker uses this weights in loss function simply multiplying it on a group summand.<br/>
So the bigger weight $\rightarrow$ the more attention for query.<br/>
Let's show an example of training procedure with random query weights.

In [19]:
def create_weights(queries):
    query_set = np.unique(queries)
    query_weights = np.random.uniform(size=query_set.shape[0])
    weights = np.zeros(shape=queries.shape)
    
    for i, query_id in enumerate(query_set):
        weights[queries == query_id] = query_weights[i]
    
    return weights
    

train_with_weights = Pool(
    data=X_train,
    label=y_train,
    group_weight=create_weights(queries_train),
    group_id=queries_train
)

test_with_weights = Pool(
    data=X_test,
    label=y_test,
    group_weight=create_weights(queries_test),
    group_id=queries_test
)

fit_model(
    'RMSE', 
    additional_params={'train_dir': 'RMSE_weigths'}, 
    train_pool=train_with_weights,
    test_pool=test_with_weights
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### A special case: top-1 prediction

Someday you may face with a problem $-$ you will need to predict the top one most relevant object for a given query.<br/>
For this purpose CatBoostRanker has a mode called __QuerySoftMax__.

Suppose our dataset contain a binary target: 1 $-$ mean best document for a query, 0 $-$ others.<br/>
We will maximize the probability of being the best document for given query.<br/>
MSRANK dataset doesn't contain binary labels, but for example of method __QuerySoftMax__ we convert it to that format,<br/> choosing a best document for every query.

In [20]:
def get_best_documents(labels, queries):
    query_set = np.unique(queries)
    num_queries = query_set.shape[0]
    by_query_arg_max = {query: -1 for query in query_set}
    
    for i, query in enumerate(queries):
        best_idx = by_query_arg_max[query]
        if best_idx == -1 or labels[best_idx] < labels[i]:
            by_query_arg_max[query] = i
    
    binary_best_docs = np.zeros(shape=labels.shape)
    for arg_max in by_query_arg_max.values():
        binary_best_docs[arg_max] = 1.
        
    return binary_best_docs

In [21]:
best_docs_train = get_best_documents(y_train, queries_train)
best_docs_test = get_best_documents(y_test, queries_test)

train_with_weights = Pool(
    data=X_train,
    label=best_docs_train,
    group_id=queries_train,
    group_weight=create_weights(queries_train)
)

test_with_weights = Pool(
    data=X_test,
    label=best_docs_test,
    group_id=queries_test,
    group_weight=create_weights(queries_test)
)

fit_model(
    'QuerySoftMax',
    additional_params={'custom_metric': 'AverageGain:top=1'},
    train_pool=train_with_weights,
    test_pool=test_with_weights
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Reducing ploblem, step 2

Now lets look at example of documents relevance:

$$ 
    \begin{align}
    labels(q_1) &= \begin{bmatrix}
           4 \\
           3 \\
           3 \\
           1
         \end{bmatrix},
    labels(q_2) &= \begin{bmatrix}
           2 \\
           1 \\
           1 \\
           0
         \end{bmatrix}
   \end{align}
$$

This means that with RMSE loss function we pay more attention to q1 than q2. 

To avoid this problem we introduce into RMSE a coefficient $c_q$ which depends only on query (and if fact equals to the mean of the difference between prediction and label).

$$\frac{1}{N}\sqrt{ \sum_q \sum_{d_{qk}} \left(f(d_{qk}) - l_{qk} - \color{red}{c_{q}} \right)^2 }$$

In [22]:
fit_model('QueryRMSE')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Reducing problem, step 3

Since the goal of ranking is to predict a list of documents (which can be generated from given document relevances) RMSE loss function doesn't take into account relations between documents: the first is better than second, second is better than third and fifth etc.

We can easily bring this information into the loss function, reducing problem not to regression but classification for two documents $(d_i, d_j)$ -- does $i$th better than $j$th or not.

So we minimize the negative loglikelihood:

$$ - \sum_{i,j \in Pairs} \log \left( \frac{1}{1 + \exp{-(f(d_i) - f(d_j))}} \right) $$

Methods based on pair comparisons called __pairwise__ in CatBoostRanker this objective called __PairLogit__.

There's no need to change the dataset CatBoost generate the pairs for us. The number of generating pairs managed via parameter max_size.

In [23]:
fit_model('PairLogit')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Also we can to specify the pairs directly. There are two ways to do that:

1. Two-dimensional matrix with shape=(num_pairs, 2) $\rightarrow$ (winner_id, loser_id): list, numpy.array, pandas.DataFrame.
2. Path two the input file that contains pair descriptions:
    * Row format: $\texttt{[winner index, loser index, pair weight]}$

In [24]:
def read_groups(file_name):
    groups = {}
    group_ids = []

    with open(file_name) as f:
        for doc_id, line in enumerate(f):
            line = line.split(',')[:2]
            
            label, query_id = float(line[0]), int(line[1])
            if query_id not in groups:
                groups[query_id] = []
            groups[query_id].append((doc_id, label))

            group_ids.append(query_id)

    return groups, group_ids
            
train_groups, train_group_ids = read_groups(train_file)
assert num_queries == len(train_groups)

In [25]:
pairs = []

for group in train_groups.values():
    for i in range(len(group)):
        for j in range(i, len(group)):
            if i == j:
                continue
            doc_i, relevance_i = group[i]
            doc_j, relevance_j = group[j]
            if relevance_i < relevance_j:
                pairs.append((doc_j, doc_i))
            else:
                pairs.append((doc_i, doc_j))
                
pairs_file = os.path.join(data_dir, 'pairs.csv')

with open(pairs_file, 'w') as f:
    for pair in pairs:
        f.write(str(pair[0]) + '\t' + str(pair[1]) + '\t1\n')

In [26]:
pool1 = Pool(data=X_train, label=y_train, group_id=train_group_ids, pairs=pairs)
pool2 = Pool(data=train_file, column_description=description_file, pairs=pairs_file, delimiter=',')

### Reducing problem, step 3.1

Thus we know that $f(d_{qk})$ is a ensemble of trees, we can accurately solve the minimization task from step 3.

This method called __PairLogitPairwise__.

In [27]:
fit_model('PairLogitPairwise')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Reducing problem, step 4

Previous loss function directly minimize the number of pairs $(d_i, d_j)$ where $l_i > l_j$ but $f(d_i) < f(d_j)$, simply said the number of incorrectly placed documents.

Since the user attention is high on the first documents and low on last the incorrect switch of the first two documents and last two has different cost.

In steps 3 and 3.1 user can set the weight for pair.

Method __YetiRank__ take this effect into account and generates weights for pairs according to their positions ([paper](https://cache-mskstoredata08.cdn.yandex.net/download.yandex.ru/company/to_rank_challenge_with_yetirank.pdf)).

$$ - \sum_{i,j \in Pairs} \color{red}{w_{ij}} \log \left( \frac{1}{1 + \exp{-(f(d_i) - f(d_j))}} \right) $$

In [28]:
fit_model('YetiRank')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Step 4.1

As in step 3.1 __YetiRankPairwise__ is slower than __YetiRank__, but gives more accurate results.

In [29]:
fit_model('YetiRankPairwise')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [30]:
widget = MetricVisualizer(['RMSE', 'QueryRMSE', 'PairLogit', 'PairLogitPairwise', 'YetiRank', 'YetiRankPairwise'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Simple classification

Very fast $\rightarrow$ very slow; Simple method $\rightarrow$ complex method; Low quality $\rightarrow$ high quality.

1. RMSE
2. QueryRMSE
3. PairLogit
4. PairLogitPairwise
5. YetiRank
6. YetiRankPairwise

Besides our classification, the quality of the concrete method may depend on your dataset.

Look on NDCG metric of method YetiRank $-$ it's underfitted.

In [31]:
fit_model('YetiRank', {'train_dir': 'YetiRank-lr-0.3', 'learning_rate': 0.3})

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [32]:
widget = MetricVisualizer(['YetiRank', 'YetiRank-lr-0.3'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Additional parameters

__Metric period__

Period in iterations of calculation metrics. This parameter can speed up training process.

In [33]:
fit_model('YetiRank', {'metric_period': 50})

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

__Task type__

You can significantly speed up training procedure switching to gpu.

In [34]:
fit_model('YetiRank', {'task_type': 'GPU'})

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CatBoostError: catboost/libs/train_lib/trainer_env.cpp:9: Environment for task type [GPU] not found